In [11]:
#完整前向传播样例（图3.6）
import tensorflow as tf
#1.定义神经网络框架：输入，输出，中间结果，参数（权值）
#x = tf.constant([[0.7,0.9]])#x是一个2*1的矩阵,这样效率低
#用placeholder机制提供输入数据：
#x = tf.placeholder(tf.float32,shape = (1,2),name = 'input')
##用placeholder机制提供多个样例输入数据，如3*2
x = tf.placeholder(tf.float32,shape = (3,2),name = 'input')

#3*2矩阵：tf.constant(-1.0, shape=[2, 3]) => [[-1. -1. -1.]
                                    #[-1. -1. -1.]]
#定义参数
w1 = tf.Variable(tf.random_normal((2,3),stddev = 1,seed = 1))#随机生成正态3*2矩阵变量参数
w2 = tf.Variable(tf.random_normal((3,1),stddev = 1,seed = 1))#1*3矩阵
#定义中间结果和输出
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

#2.会话启动图
sess = tf.Session()

#变量需要初始化
#sess.run(w1.initializer)
#sess.run(w2.initializer)

#3.当变量很多的时候可以用一句话初始化所有的变量
init_op = tf.global_variables_initializer()
sess.run(init_op)
print (sess.run(w1))
print (sess.run(w2))
#如果不用placeholder输入机制就运行以下
#print(sess.run(y))

#4.如果用placeholder就需要用字典指定x的取值
print(sess.run(y,feed_dict ={x:[[0.7,0.9],[0.1,0.4],[0.5,0.8]]}))

#tensorboard可视化，需要输入命令tensorboard --logdir logs
writer = tf.summary.FileWriter("logs/", sess.graph)
#5.关闭对话
sess.close()



[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
[[3.957578 ]
 [1.1537654]
 [3.1674924]]


In [19]:
#完整神经网络样例：
import tensorflow as tf
from numpy.random import RandomState
#反向传播算法步骤：
#1.定义batch大小.2.计算前向传播算法得到y。3.计算cost function（y'-y）.
#4.根据梯度下降法（需要求出cost对w的偏导），计算反向传播(更新w'=w-l*梯度)。5.更新完w'，然后观察cost，若cost=0,结束。
#若cost!=0,但是达到训练次数，结束。若cost!=0,没达到训练次数，下一个batch+1

#Part A:定义
#1.定义batch大小
batch_size = 8

#2.定义神经网络框架：输入，输出，中间结果，参数（权值），激活函数
#输入：前向+反向
x = tf.placeholder(tf.float32,shape = (None,2),name = 'x-input')#None可以为多个任何size的样例输入
y_ = tf.placeholder(tf.float32,shape = (None,1),name = 'y-input')#正确结果
#输出和中间结果:
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)#预测结果
#参数
w1 = tf.Variable(tf.random_normal((2,3),stddev = 1,seed = 1))#随机生成正态3*2矩阵变量参数
w2 = tf.Variable(tf.random_normal((3,1),stddev = 1,seed = 1))#1*3矩阵
#定义激活函数
y = tf.sigmoid(y)
#3.定义损失函数
cross_entropy = -tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0))+(1-y)*tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
#4.定义反向传播算法：常用的优化方法有三种： tf.train.GradientDescentOptimizer,tf.train.AdamOptimizer和tf.train.MomentumOptimizer
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)#用优化方法最小化损失函数

#Part B:随机生成一个模拟数据集
rdm = RandomState(1)#初始化随机值为1，随机生成一个整数
dataset_size = 128
X = rdm.rand(dataset_size,2)#随机生成2*128的矩阵
Y = [[int(x1+x2<1)] for (x1,x2) in X]#new_list = [expression(i) for i in old_list if filter(i)]
#expression(i)是[int(x1+x2<1)],如果x1+x2<1为0，如果x1+x2>1为1，这样就得到一个由1和0组成的list Y

#Part C:创建对话
#with tf.Session() as sess:#用with as语句简化代码,这样就可以不用调用Sess.close()函数关闭对话。
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)
#打印训练之前的参数    
print (sess.run(w1))
print (sess.run(w2))

#训练过程

#分批次feed数据
STEPS = 5000
for i in range(STEPS):
    start = (i*batch_size) % dataset_size
    end = min(start+batch_size,dataset_size)
    
#根据选取的样本训练网络并更新参数。反向传播train_step做的事情：
#1.计算前向传播算法得到y。2.计算cost function（y'-y）.
#3.根据梯度下降法（需要求出cost对w的偏导），计算反向传播(更新w'=w-l*梯度)。4.更新完w'，然后观察cost，若cost=0,结束。
#若cost!=0,但是达到训练次数，结束。若cost!=0,没达到训练次数，下一个batch+1

    sess.run(train_step,
             feed_dict={x : X[start:end],y_ : Y[start:end]})
    
# 每隔一段时间计算所有数据上的交叉熵   
    if i % 1000 == 0:
        total_cross_entropy = sess.run(cross_entropy,
                                       feed_dict={x: X,y_:Y})
        print("After %d training step(s),cross entropy on all data is %g"%(i,total_cross_entropy)) 
print (sess.run(w1))
print (sess.run(w2))

#关闭对话
sess.close()


[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s),cross entropy on all data is 0.314006
After 1000 training step(s),cross entropy on all data is 0.0684551
After 2000 training step(s),cross entropy on all data is 0.033715
After 3000 training step(s),cross entropy on all data is 0.020558
After 4000 training step(s),cross entropy on all data is 0.0136867
[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.0992484   0.5912243 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
